# 単位ベクトル化
- 手法
    - 固有名詞ベクトルを単位ベクトル化
    - 単位ベクトルに対してDBSCANでクラスタリング
    - クラスタの重心とoutlaierを取得

- 目的
    - 固有名詞のダブリを除去

In [9]:
from wordcloud import WordCloud
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib 
from scipy import optimize
from scipy import stats

import warnings
warnings.filterwarnings('ignore') # warningが出ないように設定
pd.set_option("display.max_rows", None) # pandasの表示上限をなくす
pd.set_option("display.max_columns", None) # pandasの表示上限をなくす
import pickle
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials
import collections
import gensim

model = gensim.models.Word2Vec.load("../latest-ja-word2vec-gensim-model/word2vec.gensim.model")

In [10]:
f = open('../koyuu_noun_sample_list.binaryfile','rb')
sample_noun_vec_list = pickle.load(f)

In [11]:
# 単位ベクトル化
sample_nomal_vec_list = []

for v in sample_noun_vec_list:
    a = np.array(v)
    x = np.linalg.norm(a)
    b = a / x
    sample_nomal_vec_list.append(b)

In [12]:
len(sample_nomal_vec_list)

43349

In [13]:
f = open("../0_sample_nomal_vec_list.binalyfile",'wb')
pickle.dump(sample_nomal_vec_list,f)
f.close

<function BufferedWriter.close>

In [14]:
f = open('../0_sample_nomal_vec_list.binalyfile','rb')
sample_noun_vec_list = pickle.load(f)
len(sample_noun_vec_list)

43349

In [ ]:
# 密なクラスタを算出

eps_minPts = {}
minPts_data = {}

print("noun_vec_list数 : ", len(sample_noun_vec_list))

for eps in range(1,10,1):
    eps = eps/10
    
    for minPts in range(2,12, 2):
        dbscan = DBSCAN(eps=eps,min_samples=minPts).fit(sample_noun_vec_list)
        
        y_dbscan = dbscan.labels_
        
        print("y_dbscan", y_dbscan)
        
        print("eps:",eps,",minPts:", minPts)
        
        # outlier数
        print("outlier数", len(np.where(y_dbscan ==-1)[0]))
        
        # クラスタ数
        print("クラスタ数", np.max(y_dbscan) +1)
        
        print("                            ")
        
        minPts_data[minPts] = y_dbscan
        eps_minPts[eps] = minPts_data
        
    # DBSCANの結果を保存
    f_name = str(eps) + "_" + str(minPts) + "_dic.binaryfile"
    save_at = "../dbscan_koyuu_nomal_noun_fold/" + f_name
    print(save_at)
    f = open(save_at,'wb')
    pickle.dump(eps_minPts,f)
    f.close
    
    eps_minPts = {}
    minPts_data = {}
    break